In [1]:
from NN_Thesis.nn_classes import *
from NN_Thesis.trainer import *
from NN_Thesis.adapters import *
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
from torch import optim


from torchvision.transforms import transforms
import numpy as np
import os
from PIL import Image
import random

from matplotlib import pyplot as plt



In [2]:
from NN_Thesis.models import *

In [3]:
torch.manual_seed(123)
random.seed(123)
batch_size = 16


from NN_Thesis.dataset import cifar_n_dataset
from torch.utils.data import DataLoader

train_path = 'data/cifar_5/cifar_04/train/data'
test_path ='data/cifar_5/cifar_04/test/data'

train_data = cifar_n_dataset(train_path)
test_data = cifar_n_dataset(train_path)
print(train_data.data.shape)

def normalize_channels(data):
    #We have a nxCxWxH array
    d = data
    d = torch.flatten(data,2,-1).to(dtype = torch.float32)/255
    mean= torch.mean(d,dim = [0,2])
    std = torch.std(d,dim = [0,2])
    return mean,std

mean,std = normalize_channels(train_data.data)

train_transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean, std),
])


test_transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_data.transform = train_transform
test_data.transform = test_transform



torch.Size([25000, 3, 32, 32])


In [4]:
train_dataloader = DataLoader(train_data,batch_size =64,shuffle = True)

x = train_data[0]
img,label = next(iter(train_dataloader))

img.shape


torch.Size([64, 3, 32, 32])

# Code For Initial Cifar5 Training

In [5]:
train_dataloader = DataLoader(train_data,batch_size =32,shuffle = True)
test_dataloader = DataLoader(test_data,batch_size =32)

classes = tuple(train_data.label_names)

BNN_resnet18 = resnet_binary(num_classes = 5 , depth = 18, dataset = 'cifar10')
resnet18 = resnet(num_classes = 5 , depth = 18, dataset = 'cifar10')


BNN_trainer = Trainer(BNN_resnet18,name = 'BinaryCifar5',classes = classes,seed = 123,binarise = True)
resnet_trainer =Trainer(resnet18,name = 'Cifar5',classes = classes,seed = 123)

#Set Training Params
for trainer in [BNN_trainer]:
    trainer.lr = 0.1
    trainer.epochs = 200
    trainer.train(train_dataloader,test_dataloader)


start
start_epoch : 0
lr : 0.1
batch_size : 16
epochs : 200
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 1.330 Epoch Time 0.48 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\BinaryCifar5_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\BinaryCifar5_latest.pth saved!
Overall Accuracy : 48.7%
best_acc.pth saved!
epoch: 2 average loss: 1.188 Epoch Time 0.85 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\BinaryCifar5_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\BinaryCifar5_latest.pth saved!
Overall Accuracy : 51.6%
best_acc.pth saved!
epoch: 3 average loss: 1.176 Epoch Time 1.23 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\BinaryCifar5_best_loss.pth saved!
c:\Users\John Su\Downl

# Code For Finetuning

In [4]:
train_path = 'data/cifar_5/cifar_59/train/data'
test_path ='data/cifar_5/cifar_59/test/data'

train59_data = cifar_n_dataset(train_path)
test59_data = cifar_n_dataset(train_path)
# print(train_data.data.shape)

def normalize_channels(data):
    #We have a nxCxWxH array
    d = data
    d = torch.flatten(data,2,-1).to(dtype = torch.float32)/255
    mean= torch.mean(d,dim = [0,2])
    std = torch.std(d,dim = [0,2])
    return mean,std

mean,std = normalize_channels(train59_data.data)

train59_transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean, std),
])


test59_transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train59_data.transform = train_transform
test59_data.transform = test_transform
train59_loader = DataLoader(train59_data,batch_size=64,shuffle=True)
test59_loader = DataLoader(test59_data,batch_size=64,shuffle=True)
classes = tuple(train59_data.label_names)
classes

('dog', 'frog', 'horse', 'ship', 'truck')

## Feature Extraction

In [8]:

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
BinCifar5_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)

feat_extr_model = resnet18_adapt(num_classes= 5)
feat_extr_model.load_state_dict(BinCifar5_state_dict)
feat_extr_model.freeze()
feat_extr_model.fc = BinarizeLinear(64*5,5)

feat_ex = Trainer(feat_extr_model,name = 'feat_extract',classes = classes,seed = 123,binarise = True)
feat_ex.lr = 0.001
feat_ex.epochs = 20
feat_ex.train(train59_loader,test59_loader)




start
start_epoch : 0
lr : 0.001
batch_size : 16
epochs : 20
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 1.115 Epoch Time 0.14 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\feat_extract_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\feat_extract_latest.pth saved!
Overall Accuracy : 72.4%
best_acc.pth saved!
epoch: 2 average loss: 0.888 Epoch Time 0.29 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\feat_extract_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\feat_extract_latest.pth saved!
Overall Accuracy : 72.7%
best_acc.pth saved!
epoch: 3 average loss: 0.836 Epoch Time 0.43 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\feat_extract_best_loss.pth saved!
c:\Users\John Su\Down

In [9]:

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
BinCifar5_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)

finetune_model = resnet18_adapt(num_classes= 5)
finetune_model.load_state_dict(BinCifar5_state_dict)
finetune_model.freeze()
finetune_model.fc = BinarizeLinear(64*5,5)

finetune = Trainer(finetune_model,name = 'finetune',classes = classes,seed = 123,binarise = True)
finetune.lr = 0.01
finetune.epochs = 20
m = finetune.model
m.to(finetune.device)
for layer in [m.fc,m.bn3,m.bn3]:
    for p in layer.parameters():
        p.requires_grad = True
    layer.train()
finetune.train(train59_loader,test59_loader)


#FineTune
finetune.epochs = 100
finetune.lr = 0.1
finetune.optimizer = optim.SGD(finetune.model.parameters(), lr=finetune.lr, momentum=0.9)
finetune.scheduler = optim.lr_scheduler.CosineAnnealingLR(finetune.optimizer,finetune.epochs)
m.unfreeze()
finetune.train(train59_loader,test59_loader)

start
start_epoch : 0
lr : 0.01
batch_size : 16
epochs : 20
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 0.881 Epoch Time 0.14 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_latest.pth saved!
Overall Accuracy : 73.7%
best_acc.pth saved!
epoch: 2 average loss: 0.756 Epoch Time 0.29 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_latest.pth saved!
Overall Accuracy : 73.5%
epoch: 3 average loss: 0.748 Epoch Time 0.43 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\finetune_la

# Adapter FineTune

## Single Autoencoder After bn2 

In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)

bottleneck = autoencoder_adapter(320,200)
adapt_fine = resnet18_adapt(num_classes=5)
adapt_fine.load_state_dict(Bin_state_dict)
adapt_fine.freeze()
adapt_fine.add_adapter(after = 'bn2',adapter = bottleneck)

adapt_fine_trainer = adapter_Trainer(model = adapt_fine,seed = 123,name = 'adapt_fine',classes = classes,binarise= True)
for trainer in [adapt_fine_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =70
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 70
    # trainer.scheduler.T_max= trainer.T_max
    # trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.CosineAnnealingLR(trainer.optimizer, T_max= trainer.epochs)
    trainer.train(train59_loader,test59_loader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 70
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 1.388 Epoch Time 0.14 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adapt_fine_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adapt_fine_latest.pth saved!
Overall Accuracy : 44.1%
best_acc.pth saved!
epoch: 2 average loss: 1.144 Epoch Time 0.28 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adapt_fine_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adapt_fine_latest.pth saved!
Overall Accuracy : 52.3%
best_acc.pth saved!
epoch: 3 average loss: 1.068 Epoch Time 0.42 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adapt_fine_best_loss.pth saved!
c:\Users\John Su\Downloads\Sydn

## 2 Adapters


In [20]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)

# bottleneck = autoencoder_adapter(320,200)
conv_bn = conv_adapter(80,kernel = 1, padding= 0)

adapt_fine2 = resnet18_adapt(num_classes=5)
adapt_fine2.load_state_dict(Bin_state_dict)
adapt_fine2.freeze()

# adapt_fine2.add_adapter(after = 'bn2',adapter = bottleneck)
adapt_fine2.add_adapter(after = 'layer1',adapter = conv_bn)

adapt_fine2_trainer = adapter_Trainer(model = adapt_fine2,seed = 123,name = 'adapt_fine2',classes = classes,binarise= True)
for trainer in [adapt_fine2_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 32
    trainer.epochs =50
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 50
    trainer.scheduler = optim.lr_scheduler.CosineAnnealingLR(trainer.optimizer, T_max= trainer.epochs)
    trainer.train(train59_loader,test59_loader)

start
start_epoch : 0
lr : 0.01
batch_size : 32
epochs : 50
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([64, 320, 8, 8])
torch.Size([64, 80, 32, 32])
torch.Size([64, 160, 16, 16])
torch.Size([6

KeyboardInterrupt: 

# Weight Comparison


In [55]:
def compare_weights(main_model:nn.Module,*models:nn.Module):

    main_weights = dict()
    for (name, param) in main_model.named_parameters():
        main_weights[name] = param

    n = len( list(main_model.named_parameters()))
    print(n)
    
    for model in models:
        i = 0
        for name, param in model.named_parameters():
            if name in main_weights.keys():
                main_weight = main_weights[name]
                if not (torch.all(main_weight == param)):
                    print(f'parameters {name} in main_model and other model do not match')
            else:
                print(f'name {name} does not exist in original model')

In [56]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)
fineTune_dict = torch.load('adapt_fine2_best_acc.pth',map_location= device)

og_model =  resnet18_adapt(num_classes=5)
og_model.load_state_dict(Bin_state_dict)
og_model.to(device = 'cpu')




finetune = resnet18_adapt(num_classes=5)

finetune.to(device = 'cpu')
bottleneck = autoencoder_adapter(320,200)
conv_bn = conv_bottleneck_adapter(80,40,kernel = 3, padding= 1)
finetune.add_adapter(after = 'bn2',adapter = bottleneck)
finetune.add_adapter(after = 'layer1',adapter = conv_bn)

finetune.load_state_dict(fineTune_dict)

# h = resnet18_adapt(num_classes=5)
# x = resnet18_adapt(num_classes=5)
compare_weights(og_model,finetune)


# h = resnet18_adapt(num_classes=5)
# x = resnet18_adapt(num_classes=5)

# print(torch.all(h.weight == x.weight))

# compare_weights(h,x)
# print(h.weight)
# adapt_fine2.conv1.weight

51
name layer1.2.conv1.weight does not exist in original model
name layer1.2.bn1.weight does not exist in original model
name layer1.2.bn1.bias does not exist in original model
name layer1.2.deconv1.weight does not exist in original model
name layer1.2.bn2.weight does not exist in original model
name layer1.2.bn2.bias does not exist in original model
parameters bn3.weight in main_model and other model do not match
parameters bn3.bias in main_model and other model do not match
parameters fc.weight in main_model and other model do not match
name adapter_dict.bn2,autoencoder_adapter,0.l_in.weight does not exist in original model
name adapter_dict.bn2,autoencoder_adapter,0.l_in.bias does not exist in original model
name adapter_dict.bn2,autoencoder_adapter,0.bn1.weight does not exist in original model
name adapter_dict.bn2,autoencoder_adapter,0.bn1.bias does not exist in original model
name adapter_dict.bn2,autoencoder_adapter,0.l_out.weight does not exist in original model
name adapter_di

In [15]:
# from NN_Thesis.models.binarized_modules import BinarizeConv2d
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

bt_model = resnet18_adapt()
bt_model.load_state_dict(Bin_state_dict)

conv_model = resnet18_adapt()
conv_model.load_state_dict(Bin_state_dict)

conv_bt_model = resnet18_adapt()
conv_bt_model.load_state_dict(Bin_state_dict)



adapters = [
    (bottleneck_adapter((80,32,32),5),bottleneck_adapter((160,16,16),5),bottleneck_adapter((320,8,8),5)),
    (conv_adapter(80,kernel= 3,padding = 1),conv_adapter(160,kernel= 3,padding = 1),conv_adapter(320,kernel=1)),
    (conv_bottleneck_adapter(80,80,kernel=3,padding=1),conv_bottleneck_adapter(160,80,kernel=3,padding=1),conv_bottleneck_adapter(320,160,kernel=3,padding=1))
]


#Set all layer requires grad to false and set to eval mode()
for m,adpts in zip([bt_model,conv_model,conv_bt_model],adapters):
    m.freeze()
    m.add_adapter(after = 'layer1',adapter =adpts[0])
    m.add_adapter(after = 'layer2',adapter =adpts[1])
    m.add_adapter(after = 'layer3',adapter =adpts[2])


# bottle_trainer = adapter_Trainer(model = bt_model,seed = 123,name = 'Bottleneck_BNN2',classes = classes,binarise= True)
conv_trainer = adapter_Trainer(model = conv_model,seed = 123,name = 'Conv_BNN2',classes = classes,binarise= True)
conv_bt_trainer = adapter_Trainer(model = conv_bt_model,seed = 123,name = 'ConvBN_BNN2',classes = classes,binarise= True)
# trainer.load('test_150.pth',map_location= device,load_model= True)
for trainer in [conv_trainer,conv_bt_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =80
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 80
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 2.592 Epoch Time 0.89 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_latest.pth saved!
Overall Accuracy : 29.9%
best_acc.pth saved!
epoch: 2 average loss: 1.996 Epoch Time 1.77 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_latest.pth saved!
Overall Accuracy : 26.5%
epoch: 3 average loss: 1.882 Epoch Time 2.64 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\Conv_BNN2_best_loss.pth saved!
c:\Users\John Su\Downloads\S

# DEBUGGING TIME


In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

debug = resnet18_adapt()
debug.load_state_dict(Bin_state_dict)
debug.freeze()
debug.add_adapter(after = 'layer3',adapter = identity_adapter())

debug_trainer = adapter_Trainer(model = debug,seed = 123,name = 'debug',classes = classes,binarise= True)
for trainer in [debug_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =10
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 10
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 0.448 Epoch Time 0.49 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 2 average loss: 0.447 Epoch Time 0.98 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 3 average loss: 0.445 Epoch Time 1.47 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\debug_best_loss.pth saved!
c:\Users\John Su\Downloads\S

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('test_best_acc.pth',map_location= device)

adapter = mini_bottleneck_adapter()

mini_adapt_model = resnet18_adapt()
mini_adapt_model.load_state_dict(Bin_state_dict)
mini_adapt_model.freeze()
mini_adapt_model.add_adapter(after = 'layer3',adapter = adapter)

mini_adapt_trainer = adapter_Trainer(model = mini_adapt_model,seed = 123,name = 'mini_adapt',classes = classes,binarise= True)
for trainer in [mini_adapt_trainer]:
    m = trainer.model
    m.to(trainer.device)
    for layer in [m.fc,m.bn3,m.bn3]:
        for p in layer.parameters():
            p.requires_grad = True
        layer.train()
    trainer.lr = 0.01
    trainer.batch_size = 128
    trainer.epochs =10
    trainer.epoch_chkpts = []
    trainer.start_epoch = 0
    trainer.T_max = 80
    # trainer.scheduler.T_max= trainer.T_max
    trainer.optimizer = optim.Adam(trainer.model.parameters(), lr=trainer.lr)
    trainer.scheduler = optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.95)
    trainer.train(trainloader,testloader)

start
start_epoch : 0
lr : 0.01
batch_size : 128
epochs : 10
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 0.447 Epoch Time 0.56 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_latest.pth saved!
Overall Accuracy : 83.6%
best_acc.pth saved!
epoch: 2 average loss: 0.447 Epoch Time 1.12 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_latest.pth saved!
Overall Accuracy : 83.7%
best_acc.pth saved!
epoch: 3 average loss: 0.444 Epoch Time 1.69 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\mini_adapt_best_loss.pth saved!
c:\

# Adapter Location

In [8]:

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Bin_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)


layers = ['layer1','layer2','layer3']
channels = [80,160,320]

Bin_state_dict = torch.load('BinaryCifar5_best_acc.pth',map_location= device)
for layer,channel in zip(layers,channels):
    for t in ['adp_only','full_ft']:
        print(t,layer,channel)
        torch.manual_seed(42)
        adapter_model = resnet18_adapt(num_classes=5)
        adapter_model.load_state_dict(Bin_state_dict)
        
        if t == 'adp_only':
            adapter_model.freeze()
        conv_adp = conv_adapter(channel,kernel = 1, padding= 0)
        adapter_model.add_adapter(after = layer,adapter = conv_adp)
        trainer = adapter_Trainer(model = adapter_model,seed = 123,name = f'adp_{layer}_{t}',classes = classes,binarise= True)
        m = trainer.model
        m.to(trainer.device)
        for head in [m.fc,m.bn3,m.bn3]:
            for p in head.parameters():
                p.requires_grad = True
            head.train()
        trainer.lr = 0.01
        trainer.batch_size = 32
        trainer.epochs =1
        trainer.epoch_chkpts = []
        trainer.start_epoch = 0
        trainer.T_max = 50
        trainer.scheduler = optim.lr_scheduler.CosineAnnealingLR(trainer.optimizer, T_max= trainer.epochs)
        trainer.train(train59_loader,test59_loader)

adp_only layer1 80
start
start_epoch : 0
lr : 0.01
batch_size : 32
epochs : 1
epoch_chkpts : []
device : cuda:0
criterion : CrossEntropyLoss()
optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
seed : 123
epoch: 1 average loss: 1.038 Epoch Time 0.23 mins
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adp_layer1_adp_only_best_loss.pth saved!
c:\Users\John Su\Downloads\SydneyUni\thesis\Thesis\adp_layer1_adp_only_latest.pth saved!
Overall Accuracy : 71.0%
best_acc.pth saved!
Data Saved to adp_layer1_adp_only.csv
Finished Training: 
Total Time 0.003774 hours
 Average Time Per Epoch 13.59 seconds
full_ft BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) 80


TypeError: Please ensure that both "after" and "adapter" are both list like and the same length

In [16]:
b = resnet18_adapt(num_classes=5)

b.eval()
x = torch.rand((1,3,32,32))

b(x)

torch.Size([1, 80, 32, 32])
torch.Size([1, 160, 16, 16])
torch.Size([1, 320, 8, 8])


tensor([[  0.0000, -19.9601, -34.0094, -21.9788, -34.0053]],
       grad_fn=<LogSoftmaxBackward0>)